The World Bank WDI Indicators is a quite a large dataset with 100's of indicators. This code is designed to you to specify an indicators list and download on these indicators. This i found is a more efficent way to download data. One issue i ran into was that some indicator codes may not be avialble with due to code changes etc. The code therefore assess which indicators are available and download these while point out which downloads failed. 

In [ ]:
import wbdata
import pandas as pd
from datetime import datetime
import os
import tkinter as tk
from tkinter import filedialog


def get_file_path(title, action="open", default_extension=".xlsx"):
    root = tk.Tk()
    root.withdraw()
    if action == "open":
        return filedialog.askopenfilename(
            title=title,
            filetypes=[("Excel files", "*.xlsx *.xls")]
        )
    else:
        return filedialog.asksaveasfilename(
            title=title,
            defaultextension=default_extension,
            filetypes=[("Excel files", "*.xlsx")]
        )

def download_indicators(indicators_dict, countries, dates):
    """Try to download each indicator and return successful ones"""
    available_data = []
    failed_indicators = []
    
    # Try each indicator individually
    for code, name in indicators_dict.items():
        try:
            indicator = {code: name}
            df = wbdata.get_dataframe(indicator, country=countries, date=dates)
            if not df.empty:
                available_data.append(df)
            else:
                failed_indicators.append((code, name))
        except:
            failed_indicators.append((code, name))
            print(f"Could not download indicator: {code} - {name}")
            continue
    
    # Combine all successful downloads if any
    if available_data:
        return pd.concat(available_data, axis=1), failed_indicators
    else:
        return None, failed_indicators
    
# Prompt user to select files
print("Please select the countries Excel file...") # this is included in the Github repo. Its set up to download Caribbean countries. Please change as needed
countries_file = get_file_path("Select Countries Excel File")
print("Please select the indicators Excel file...")
indicators_file = get_file_path("Select Indicators Excel File") # this is also included in the Github repo. You can change as needed. 

# Read the Excel files
countries_df = pd.read_excel(countries_file)
indicators_df = pd.read_excel(indicators_file)

# Convert countries to list
countries = countries_df.iloc[:, 0].tolist()  # Take first column

# Convert indicators to dictionary
requested_indicators = dict(zip(indicators_df['indicator_code'], indicators_df['indicator_name']))


# Specify dates
dates = (datetime(2000, 1, 1), datetime(2029, 1, 1)) # change this as needed

print("\nAttempting to download data...")
wb_data, failed_indicators = download_indicators(requested_indicators, countries, dates)

if failed_indicators:
    print("\nThe following indicators could not be downloaded:")
    for code, name in failed_indicators:
        print(f"- {code}: {name}")

if wb_data is not None:
    # Clean data
    wb_data = wb_data.reset_index()
    


    # Prompt for saving the output
    print("Please select where to save the output file...")
    save_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                            filetypes=[("CSV files", "*.csv")])
    
    # Export to CSV
    if save_path:
        wb_data.to_csv(save_path, index=False)
        print(f"Data exported to: {save_path}")
    
    print("\nFirst few rows of downloaded data:")
    print(wb_data.head())
else:
    print("\nNo data could be downloaded. Please check your indicator codes.")